In [1]:
import pandas as pd
import numpy as np

In [2]:
true=pd.read_csv('Data/True.csv')

In [3]:
true.shape

(21417, 4)

In [4]:
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [5]:
fake=pd.read_csv('Data/Fake.csv')

In [6]:
fake.shape

(23481, 4)

In [7]:
true['label']=1
fake['label']=0

In [8]:
true_sub=true.iloc[:20000,:]
true_sub.shape

(20000, 5)

In [9]:
fake_sub=fake.iloc[:20000,:]
fake_sub.shape

(20000, 5)

In [10]:
df=true_sub.append(fake_sub, ignore_index=True)
df

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1
...,...,...,...,...,...
39995,POWERFUL! HE BECAME A POLICE OFFICER AFTER Wat...,Dallas Police Officer Dan Cincinnatus wrote an...,left-news,"Sep 10, 2016",0
39996,WHY IS THE MEDIA HIDING This Endorsement?…KKK ...,It s time for the media to start doing their j...,left-news,"Sep 10, 2016",0
39997,IF HILLARY IS ELECTED And Becomes Too Sick To ...,The applications for White House interns just ...,left-news,"Sep 10, 2016",0
39998,BLACK RESIDENTS NOT HAPPY After Street Artist ...,The Obama s have convinced Black communities a...,left-news,"Sep 9, 2016",0


In [11]:
df.isnull().sum().sort_values(ascending=False)

title      0
text       0
subject    0
date       0
label      0
dtype: int64

In [12]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [13]:
import re
import nltk
corpus=[]
for i in range(0,len(df)):
  review = re.sub('[^a-zA-Z]', ' ', df['text'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if word not in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=20000, ngram_range=(1,3))

In [15]:
X=tfidf.fit_transform(corpus)
Y=df['label']

In [16]:
X_arr=X.toarray()

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_arr, Y, test_size=0.2, random_state=0)

In [18]:
from sklearn.linear_model import PassiveAggressiveClassifier
classifier = PassiveAggressiveClassifier(max_iter=1000)

In [19]:
from sklearn import metrics
classifier.fit(X_train, y_train)
pred = classifier.predict(X_test)
score = metrics.accuracy_score(y_test, pred)
print("accuracy:   %0.3f" % score)

accuracy:   0.995


In [20]:
review = re.sub('[^a-zA-Z]', ' ', fake['text'][20000])
review = review.lower()
review = review.split()    
review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
review = ' '.join(review)
review

'warn watch video encount strong languag even stronger stupid femal sailor post video sit morn color protest suppos racism nation anthem disciplin u navi disciplinari action toward femal sailor ongo spokeswoman u navi told militari time alway train make sure sailor know pro con use social media must alway observ appropri conduct alway subject ucmj time ad nation anthem basic say land free home brave except hirel slave support anyth like femal sailor said video post facebook continu think colin realli good point said bodi street'

In [21]:
val = tfidf.transform([review]).toarray()

In [22]:
classifier.predict(val)

array([0], dtype=int64)

In [23]:
import pickle

In [24]:
pickle.dump(classifier, open('model.pkl', 'wb'))

In [25]:
pickle.dump(tfidf, open('tfidfvect.pkl', 'wb'))